In [5]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn import feature_selection as f_select
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
import patsy

%matplotlib inline

In [6]:
file_path = 'transfermarkt/tables.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['Table','Position','Club','Club Short','GP','Wins','Draws','Losses','GFandGA','GD','Points']
df = pd.DataFrame(data,columns=labels)
df['GF']=df.GFandGA.str.split(':').str[0]
df['GA']=df.GFandGA.str.split(':').str[1]
df['Season']=df.Table.str.strip().str[-5:]
df = df.convert_objects(convert_numeric=True)
df['MaxP']=df.GP * 3
df['CalcP']= df.Wins * 3 + df.Draws
df['League']=df.Table.str[6:].str[:-7]

/Users/stephenchou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:14: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [7]:
merge_df_columns = ['Position','Season','Club','Club Short','GF','GA','GD','MaxP','CalcP','League']
merge_columns = ['Club','Season']
merge_df = df[merge_df_columns]

In [8]:
#transfer_file_path = 'transfermarkt/transfers_old.json'
transfer_file_path = 'transfermarkt/transfers.json'
with open(transfer_file_path) as json_data_transfer:
    d2 = json.load(json_data_transfer)
    data2 = []
for row in d2:
    for k in row:
        data2.append(row[k])
        
labels_transfer = ['Category','Club','Player','LongOtherClub','OtherClub','Fee']
transfer_df = pd.DataFrame(data2,columns=labels_transfer)
transfer_df['Direction']=transfer_df['Category'].str.strip().str.split(' ').str[0]
transfer_df['Season']=transfer_df['Category'].str.strip().str.split(' ').str[1]

In [9]:
transfer_df['Num']=0
transfer_df['Num']= transfer_df['Fee'].str.replace(',','.').str.replace("'",'').str.extract('^(\d+(.\d{2,3})*)',expand=True).astype(float)

In [10]:
transfer_df=transfer_df.fillna(0)
transfer_df['Mult']=np.where(transfer_df['Fee'].str.contains('Mill').fillna(False), 1000000, np.where(transfer_df['Fee'].str.contains('Th').fillna(False),1000,0))
transfer_df['NumFee']=transfer_df.Num * transfer_df.Mult

In [11]:
#interpret end of loan and Loan as the same thing.
transfer_df['Loan']=np.where(transfer_df['Fee'].str.lower().str.contains('loan').fillna(False), 1,0)
#If not loan and NumFee = 0, then free = 1
transfer_df['Free']=np.where((transfer_df['Loan']==0) & (transfer_df['NumFee']==0), 1,0)

In [12]:
merge_t_df = transfer_df.groupby(['Club','Season','Direction'])['NumFee'].agg(['sum', 'count'])
merge_t_2= merge_t_df.unstack('Direction')
merge_t_3= merge_t_2.reset_index()
merge_t_3.columns = ['Club','Season','Arrival_Sum','Departure_Sum','Arrival_Count','Departure_Count']

merge_t2_df = transfer_df.groupby(['Club','Season','Direction'])['Loan','Free'].agg(['sum'])
merge_t3_df= merge_t2_df.unstack('Direction')
merge_t3_df= merge_t3_df.reset_index()
merge_t3_df.columns = ['Club','Season','Arrival_Loan','Departure_Loan','Arrival_Free','Departure_Free']


In [13]:
transfer_df_mr=pd.merge(merge_t_3,merge_t3_df,on=(['Club','Season']))

In [14]:
complete_df = pd.merge(merge_df, transfer_df_mr, on=(['Club','Season']), how='left')

In [15]:
complete_df[["PREV_GF", "PREV_GA", "PREV_GD","PREV_MAXP","PREV_CALCP"]] = (complete_df
                                                       .groupby(["Club"])["GF","GA","GD","MaxP","CalcP"]
                                                       .transform(lambda grp: grp.shift(1)))

In [16]:
complete_df['Arrival_Paid']= complete_df['Arrival_Count'] - complete_df['Arrival_Loan']- complete_df['Arrival_Free']
complete_df['Departure_Paid']= complete_df['Departure_Count'] - complete_df['Departure_Loan']- complete_df['Departure_Free']
complete_df['Net_Spend']= complete_df['Arrival_Sum'] - complete_df['Departure_Sum']
complete_df['Year']=(np.where(complete_df['Season'].str[0:1]=='9','19','20')+complete_df['Season'].str[:2]).astype(int)

In [17]:
complete_dna_df = complete_df.dropna()
league=patsy.dmatrix('League',data=complete_dna_df,return_type='dataframe')
#strip columns
complete_dna_df=complete_dna_df.join(league)

In [18]:
complete_dna_df.to_pickle('merged_soccer_data.pkl')